In [2]:
#import librarys

import numpy as np
import pandas as pd

In [3]:
#Read data from the file

data = pd.read_csv("data.csv")
col_names = data.columns

print(type(data))

data.head(5)

<class 'pandas.core.frame.DataFrame'>


,feature1,feature2,feature3,feature4,class
0,5.0,3.5,1.3,0.3,0
1,6.9,3.1,4.9,1.5,1
2,5.8,2.6,4.0,1.2,1
3,6.7,3.0,5.2,2.3,2
4,5.1,3.3,1.7,0.5,0


In [3]:
#Node structure of the tree we creating

class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):
        
        #decision node
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain
        
        #leaf node
        self.value = value

In [54]:
#Decision Tree


class DecisionTreeClassifier():
    def __init__(self, min_samples_split=2, max_depth=2):
        '''constructor of the class'''
        
        # initialize the root of the tree 
        self.root = None
        
        # stopping conditions
        self.min_samples_split = min_samples_split       #minimum samples required in a independent feature for a split
        self.max_depth = max_depth       #max depth of the tree
        
    def build_tree(self, dataset, depth=0):
        ''' recursive function for building the decision tree by using training dataset, we also initialized depth of tree as 0 ''' 
        
        X, Y = dataset[:,:-1], dataset[:,-1]   #separating dataset into X and Y
        num_samples, num_features = np.shape(X)

        #print('num_samples' , num_samples, 'num_features', num_features)
        
        # splitting
        if num_samples>=self.min_samples_split and depth<=self.max_depth:
            
            # find the best split
            best_split = self.get_best_split(dataset, num_samples, num_features)
            
            # check if information gain is positive
            if best_split["info_gain"]>0:
                # recursively build left sub tree
                left_subtree = self.build_tree(best_split["dataset_left"], depth+1)
                # recursively build right sub tree
                right_subtree = self.build_tree(best_split["dataset_right"], depth+1)
                # return decision node
                return Node(best_split["feature_index"], best_split["threshold"], 
                            left_subtree, right_subtree, best_split["info_gain"])
        
        # compute leaf node
        leaf_value = self.calculate_leaf_value(Y)
        # print(leaf_value)
        # return leaf node
        return Node(value=leaf_value)
    
    def get_best_split(self, dataset, num_samples, num_features):
        ''' function to find the best split, it takes input as training datset, number of samples in dataset and number of features '''
        
        # dictionary for storing the best split information
        best_split = {}
        max_info_gain = -float("inf")
        
        # loop over all the features
        for feature_index in range(num_features):
            
            feature_values = dataset[:, feature_index] # getting all the values of a feature based on index from dataset
            possible_thresholds = np.unique(feature_values) # getting how many unique values are their in that feature values
            
            # loop over all the feature values present in the data
            for threshold in possible_thresholds:
                # get current split
                dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
                dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
                
                # check if dataset_left and datset_right childs are not null
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    
                    # compute information gain using entropy
                    curr_info_gain = self.information_gain(y, left_y, right_y, "gini")
                    
                    # update the best split if needed
                    if curr_info_gain>max_info_gain:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = curr_info_gain
                        max_info_gain = curr_info_gain
                        
        # return best split
        return best_split
    
    def information_gain(self, parent, l_child, r_child, mode="entropy"):
        ''' compute information gain '''
        
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        if mode=="gini":
            gain = self.gini_index(parent) - (weight_l*self.gini_index(l_child) + weight_r*self.gini_index(r_child))
        else:
            gain = self.entropy(parent) - (weight_l*self.entropy(l_child) + weight_r*self.entropy(r_child))
        return gain
    
    def entropy(self, y):
        ''' function to compute entropy '''
        
        class_labels = np.unique(y)
        entropy = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            entropy += -p_cls * np.log2(p_cls)
        return entropy
    
    def gini_index(self, y):
        ''' function to compute gini index '''
        
        class_labels = np.unique(y)
        gini = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            gini += p_cls**2
        return 1 - gini
    
    
    def calculate_leaf_value(self, Y):
        ''' function to compute leaf node '''
        
        Y = list(Y)
        #print(Y)
        return max(Y, key=Y.count)
    
        
    def fit(self, X, Y):
        ''' Strating function to train the tree, it takes input as training data and calls build tree for building a tree with that training data '''
        
        training_dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(training_dataset)
        
    def print_tree(self, tree=None, indent=" "):
        ''' print the tree '''
        
        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)  # if it is a leaf node

        else:
            print("feature_"+str(tree.feature_index), "<=", tree.threshold, "?", tree.info_gain)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)
        
    def predict(self, X):
        ''' predict new dataset '''
        
        preditions = [self.make_prediction(x, self.root) for x in X]
        return preditions
    
    def make_prediction(self, x, tree):
        ''' predict a single data point '''
        
        if tree.value!=None: return tree.value
        feature_val = x[tree.feature_index]
        if feature_val<=tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)
    

In [55]:
X = data.iloc[:, :-1].values
Y = data.iloc[:, -1].values.reshape(-1,1)
#print(X)
#print(Y)
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2, random_state=41)

In [56]:
classifier = DecisionTreeClassifier(min_samples_split=3, max_depth=3)
classifier.fit(X_train,Y_train)
classifier.print_tree()

feature_2 <= 1.9 ? 0.33741385372714494
 left:0.0
 right:feature_2 <= 4.7 ? 0.3836661987599338
  left:feature_3 <= 1.6 ? 0.0525931336742147
    left:1.0
    right:2.0
  right:feature_3 <= 1.7 ? 0.054610733182161544
    left:feature_0 <= 6.9 ? 0.09999999999999998
        left:1.0
        right:2.0
    right:feature_2 <= 4.8 ? 0.0262345679012347
        left:2.0
        right:2.0


In [58]:
Y_pred = classifier.predict(X_test) 
from sklearn.metrics import accuracy_score
accuracy_score(Y_test, Y_pred)

0.9666666666666667

In [61]:
'''

Observations

1. If we use entropy as the split feature we are getting accuracy of testing data as 0.9
2. If we use gini index as the split feature we are getting accuracy of testing data as 0.96, this proves that gini index
   performs better than the entropy as the split feature 

'''

'\n\nObservations\n\n1. If we use entropy as the split feature we are getting accuracy of testing data as 0.9\n\n'

In [62]:
#decision tree from scikit

from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train,Y_train)

Y_prediction = clf.predict(X_test)
from sklearn.metrics import accuracy_score
accuracy_score(Y_test, Y_prediction)

0.9666666666666667

In [63]:
'''

Observations

1. Here we can see our decision tree and in built decision tree had the same accuracy when we use gini index as the split feaature in our 
   decision tree, this tells us that the scikit is also using gini index as the split feature for building decision tree as we see the accuracy is same.

'''

'\n\nObservations\n\n1. Here we can see our decision tree and in built decision tree had the same accuracy when we use ginin index as the split feaature in out \n   decision tree, this tells us that the scikit is also using gini index as the split feature as we see the accuracy is same.\n\n'

In [71]:
'''

Acknowledgements:

1. I have gone thorugh few online resources for structuring the decision tree in a good way like separating different logics
   into different functions, how to use different inbuilt librarys etc
2. Followed this video for more understanding and trying to come up with code changes,i used most of the function names and variables same as
   they are defined in a good way for understanding
   https://www.youtube.com/watch?v=sgQAhG5Q7iY
3. For building decision tree from scikit I refered geeksforgeeks for knowing the inbuilt librarys.
4. As most of the code changes here uses inbuilt librarys from numpy and pandas, so i have learnt how to use this librarys through this assignment
5. As i am new to python, i got good exposure to python and its syntaxes, as the logic is common for a decision tree, structring the code in correct format
   took lot of time, how to use nodes in python for building a tree, how to specify values for each node etc


'''

'\n\nAcknowledgements:\n\n1. I have gone thorugh few online resources for structuring the decision tree in a good way like separating different logics\n   into different functions, how to use different inbuilt librarys etc\n2. Followed this video for more understanding and trying to come up with code changes,i used most of the function names and variables same as\n   https://www.youtube.com/watch?v=sgQAhG5Q7iY\n3. For building decision tree from scikit I refered geeksforgeeks for knowing the inbuilt librarys\n\n\n'

In [65]:
'''

****************************** end of question1 ************************************

'''

'\n\n****************************** end of question1 ************************************\n\n'

In [67]:
'''

** learnings**

here i added some syntaxes which i built from scratch instead of using python librarys, how to calculate entropy and information gain

'''


from scipy.stats import entropy
import pandas as pd
dataset = pd.read_csv("data.csv")

In [70]:

import math
from sklearn.model_selection import train_test_split


#Divide dataset into left and right, such that left dataset had all attributes and right dataset had target attribute
x = dataset.drop(columns = ['class'])
y = dataset['class']

#split dataset into training and testing  (70% for training and 30% for testing)
x_train,x_test,y_train,y_test = train_test_split(x, y, test_size=0.3)


#print(x_train['feature1'].min())
#print(x_train['feature1'].max())

gap = x_train['feature1'].max() - x_train['feature1'].min()
#print(gap/3)


#calculate information gain for each attribute for finding root of decision tree

training_dataset_size = len(x_train)
testing_dataset_size = len(x_test)

t1 = y_train.value_counts()[0]
t2 = y_train.value_counts()[1]
t3 = y_train.value_counts()[2]

entropy_of_target = -((t1/105)*math.log(t1/105,2) + (t2/105)*math.log(t2/105,2) + (t3/105)*math.log(t3/105))
#print(entropy_of_target)

entropy_of_target = entropy([t1/105, t2/105, t3/105], base=2)
#print(entropy_of_target)


#decide root node

info_gain = []
for feature in x.columns:
    min = x_train[feature].min()
    max = x_train[feature].max()
    mid = (min+max)/2
    
    left = [0,0,0]
    right = [0,0,0]
    for i,j in zip(x_train[feature], y_train):
        if i < mid:
            if j == 0:
                left[0] = left[0]+1
            elif j == 1:
                left[1] = left[1]+1
            else:
                left[2] = left[2]+1
        else:
            if j == 0:
                right[0] = right[0]+1
            elif j == 1:
                right[1] = right[1]+1
            else:
                right[2] = right[2]+1
    
    
    total_left = left[0] + left[1] + left[2]        #check >0
    total_right = right[0] + right[1] + right[2]
    
    entropy_left = 0
    entropy_right = 0
    for i in range(3):
        if left[i] > 0:
            entropy_left = entropy_left + (left[i]/total_left)*math.log(left[i]/total_left,2);
        if right[i] > 0:
            entropy_right = entropy_right + (right[i]/total_right)*math.log(right[i]/total_right,2);
        
        
    
    #entropy_left = total_left/105 * ((left[0]/total_left)*math.log(left[0]/total_left,2) + (left[1]/total_left)*math.log(left[1]/total_left,2) + (left[2]/total_left)*math.log(left[2]/total_left))
    #entropy_right = total_right/105 * ((right[0]/total_right)*math.log(right[0]/total_right,2) + (right[1]/total_right)*math.log(right[1]/total_right,2) + (right[2]/total_right)*math.log(right[2]/total_right))
    
    entropy_left = -entropy_left * total_left/105;
    entropy_right = -entropy_right * total_right/105;
    information_gain = entropy_of_target - (entropy_left + entropy_right)
    print(information_gain)

0.42960493172623937
0.21862540660983565
0.7110933940492552
0.6927156936946566
